[Deep Learning for Computer Vision](https://pyimagesearch.com/deep-learning-computer-vision-python-book/) Practitioner Bundle 책 내용 요약

![cover](https://929687.smushcdn.com/2633864/wp-content/uploads/2020/02/deeplearning-200x300-1.jpg)

## 11.3 The Tiny ImageNet Challenge
- Stanford [cs231n 코스](http://cs231n.stanford.edu/2017/) 중 일부
- CNN 처음부터 만들거나, fine-tuning. (feature extraction 은 허용 안 됨)
- 64x64 이미지 500 개씩 200개 클래스
- resize 및 crop 되어 있어서 a bit more challenging

## . 11.3.1 Downloading Tiny ImageNet
- [Tiny ImageNet/ Kaggle](https://www.kaggle.com/datasets/akash2sharma/tiny-imagenet) 데이터셋 사용

## . 11.3.2 The Tiny ImageNet Directory Structure
- train 폴더 하위의 폴더명이 WordNet ID("synonym set" or "synsets")
- words.txt 파일에 WordNet ID 의 의미 있음
- wnids.txt 파일에 200개 id
- val 폴더에는 val_annotations.txt 파일에 파일이랑 WordNet ID 매핑 있음
- general purpose programming skill 도 많이 필요함

## . 11.3.3 Building the Tiny ImageNet Dataset
- `config/tiny_imagenet_config.py` : config 모아둠
- `build_tiny_imagenet.py` 구현 : 파싱 잘 해서 HDF5 dataset으로 변환

### Config

In [1]:
from os import path

# define the paths to the training and validation directories
TRAIN_IMAGES = "../input/tiny-imagenet/tiny-imagenet-200/train/"
VAL_IMAGES = "../input/tiny-imagenet/tiny-imagenet-200/val/images"

# define the path to the file that maps validation filenames to their corresponding class labels
VAL_MAPPINGS = "../input/tiny-imagenet/tiny-imagenet-200/val/val_annotations.txt"

# define the paths to the WordNet hierarchy files which are used to generate our class labels
WORDNET_IDS = "../input/tiny-imagenet/tiny-imagenet-200/wnids.txt"
WORD_LABELS = "../input/tiny-imagenet/tiny-imagenet-200/words.txt"

# since we do not have access to the testing data
# we need to take a number of images from the training data and use it instead
NUM_CLASSES = 200
NUM_TEST_IMAGES = 50 * NUM_CLASSES

# define the path to the output training, validation, and testing HDF5 files
TRAIN_HDF5 = "hdf5/train.hdf5"
VAL_HDF5 = "hdf5/val.hdf5"
TEST_HDF5 = "hdf5/test.hdf5"

# define the path to the dataset mean
DATASET_MEAN = "output/tiny-image-net-200-mean.json"

# define the path to the output directory used for storing plots, classification reports, etc.
OUTPUT_PATH = "output"
MODEL_PATH = path.sep.join([OUTPUT_PATH, "checkpoints", "epoch_70.hdf5"])
FIG_PATH = path.sep.join([OUTPUT_PATH, "deepergooglenet_tinyimagenet.png"])
JSON_PATH = path.sep.join([OUTPUT_PATH, "deepergooglenet_tinyimagenet.json"])

In [2]:
!mkdir hdf5

In [3]:
!mkdir output

## Prepare other Classs

- `HDF5DatasetWriter`: 데이터셋을 hdf5 로 저장하기 위한 h5py wrapper

In [4]:
import h5py
import os


class HDF5DatasetWriter:
    def __init__(self, dims, output_path, data_key="images", buf_size=1000):
        # check to see if the output path exists, and if so, raise an exception
        if os.path.exists(output_path):
            raise ValueError("The supplied `outputPath` already exists and cannot be overwritten."
                             "Manually delete the file before continuing.", output_path)

        # open the HDF5 database for writing and create two datasets:
        # one to store the images/features and another to store the class labels
        self.db = h5py.File(output_path, "w")
        self.data = self.db.create_dataset(data_key, dims, dtype="float")
        self.labels = self.db.create_dataset("labels", (dims[0],), dtype="int")

        # store the buffer size, then initialize the buffer itself along with the index into the datasets
        self.bufSize = buf_size
        self.buffer = {"data": [], "labels": []}
        self.idx = 0

    def add(self, rows, labels):
        # add the rows and labels to the buffer
        self.buffer["data"].extend(rows)
        self.buffer["labels"].extend(labels)

        # check to see if the buffer needs to be flushed to disk
        if len(self.buffer["data"]) >= self.bufSize:
            self.flush()

    def flush(self):
        # write the buffers to disk then reset the buffer
        i = self.idx + len(self.buffer["data"])
        self.data[self.idx:i] = self.buffer["data"]
        self.labels[self.idx:i] = self.buffer["labels"]
        self.idx = i
        self.buffer = {"data": [], "labels": []}

    def store_class_labels(self, class_labels):
        # create a dataset to store the actual class label names,
        # then store the class labels
        dt = h5py.special_dtype(vlen=str)  # `vlen=unicode` for Py2.7
        label_set = self.db.create_dataset("label_names", (len(class_labels),), dtype=dt)
        label_set[:] = class_labels

    def close(self):
        # check to see if there are any other entries in the buffer
        # that need to be flushed to disk
        if len(self.buffer["data"]) > 0:
            self.flush()

        # close the dataset
        self.db.close()


In [5]:
!pip install imutils

  Preparing metadata (setup.py) ... - done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=f069b3c39fc15bc2dabfdaba86093f7ec131630493a5d48505e7b908bf0a6997
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


### Build Tiny ImageNet

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imutils import paths
import numpy as np
import json
import cv2
import os
from tqdm.notebook import tqdm

In [7]:
%%time

# grab the paths to the training images, then extract the training class labels and encode them
train_paths = list(paths.list_images(TRAIN_IMAGES))
print(train_paths[:3])
train_labels = [p.split(os.path.sep)[-3] for p in train_paths]
print(train_labels[:3])
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
print(train_labels[:3])

['../input/tiny-imagenet/tiny-imagenet-200/train/n02437312/images/n02437312_229.JPEG', '../input/tiny-imagenet/tiny-imagenet-200/train/n02437312/images/n02437312_394.JPEG', '../input/tiny-imagenet/tiny-imagenet-200/train/n02437312/images/n02437312_15.JPEG']
['n02437312', 'n02437312', 'n02437312']
[53 53 53]
CPU times: user 500 ms, sys: 538 ms, total: 1.04 s
Wall time: 22 s


In [8]:
# perform stratified sampling from the training set to construct a testing set
split = train_test_split(train_paths, train_labels, test_size=NUM_TEST_IMAGES,
                         stratify=train_labels, random_state=42)
(train_paths, test_paths, train_labels, test_labels) = split
print(len(train_paths), train_labels.shape)
print(len(test_paths), test_labels.shape)

90000 (90000,)
10000 (10000,)


In [9]:
# load the validation filename => class from file and then
# use these mappings to build the validation paths and label lists
M = open(VAL_MAPPINGS).read().strip().split("\n")
print(M[:3])
M = [r.split("\t")[:2] for r in M]
print(M[:3])
val_paths = [os.path.sep.join([VAL_IMAGES, m[0]]) for m in M]
print(val_paths[:3])
val_labels = le.transform([m[1] for m in M])
print(val_labels[:3])

['val_0.JPEG\tn03444034\t0\t32\t44\t62', 'val_1.JPEG\tn04067472\t52\t55\t57\t59', 'val_2.JPEG\tn04070727\t4\t0\t60\t55']
[['val_0.JPEG', 'n03444034'], ['val_1.JPEG', 'n04067472'], ['val_2.JPEG', 'n04070727']]
['../input/tiny-imagenet/tiny-imagenet-200/val/images/val_0.JPEG', '../input/tiny-imagenet/tiny-imagenet-200/val/images/val_1.JPEG', '../input/tiny-imagenet/tiny-imagenet-200/val/images/val_2.JPEG']
[107 139 140]


In [10]:
# construct a list pairing the training, validation, and testing image paths
# along with their corresponding labels and output HDF5 files
datasets = [
    ("train", train_paths, train_labels, TRAIN_HDF5),
    ("val", val_paths, val_labels, VAL_HDF5),
    ("test", test_paths, test_labels, TEST_HDF5)
]

In [11]:
# initialize the lists of RGB channel averages
(R, G, B) = ([], [], [])

In [12]:
# loop over the dataset tuples
for (data_type, paths, labels, output_path) in datasets:
    # create HDF5 writer
    print("[INFO] building {}...".format(output_path))
    writer = HDF5DatasetWriter((len(paths), 64, 64, 3), output_path)

    # loop over the image paths
    for (i, (path, label)) in enumerate(zip(tqdm(paths), labels)):
        # load the image from disk
        image = cv2.imread(path)

        # if we are building the training dataset, then compute the mean of each channel in the image,
        # then update the respective lists
        if data_type == "train":
            (b, g, r) = cv2.mean(image)[:3]
            R.append(r)
            G.append(g)
            B.append(b)

        # add the image and label to the HDF5 dataset
        writer.add([image], [label])

    # close the HDF5 writer
    writer.close()

[INFO] building hdf5/train.hdf5...


  0%|          | 0/90000 [00:00<?, ?it/s]

[INFO] building hdf5/val.hdf5...


  0%|          | 0/10000 [00:00<?, ?it/s]

[INFO] building hdf5/test.hdf5...


  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
# construct a dictionary of averages, then serialize the means to a JSON file
print("[INFO] serializing means...")
D = {"R": np.mean(R), "G": np.mean(G), "B": np.mean(B)}
f = open(DATASET_MEAN, "w")
f.write(json.dumps(D))
f.close()

[INFO] serializing means...


In [14]:
!ls -al hdf5

total 10560896
drwxr-xr-x 2 root root       4096 Aug 16 17:01 .
drwxr-xr-x 4 root root       4096 Aug 16 16:47 ..
-rw-r--r-- 1 root root  983122048 Aug 16 17:03 test.hdf5
-rw-r--r-- 1 root root 8848082048 Aug 16 17:00 train.hdf5
-rw-r--r-- 1 root root  983122048 Aug 16 17:01 val.hdf5


In [15]:
ls -al output

total 12
drwxr-xr-x 2 root root 4096 Aug 16 17:03 ./
drwxr-xr-x 4 root root 4096 Aug 16 16:47 ../
-rw-r--r-- 1 root root   75 Aug 16 17:03 tiny-image-net-200-mean.json


In [16]:
import h5py

for _, _, _, hdf5_path in datasets:
    db = h5py.File(hdf5_path, 'r')
    print(db['images'].shape)
    db.close()


(90000, 64, 64, 3)
(10000, 64, 64, 3)
(10000, 64, 64, 3)
